In [1]:
import pandas as pd
import datetime

# ignore default warning when setting dataframe value to value in a copy
pd.options.mode.chained_assignment = None

# read in results from scrapy spiders as three new dataframes
dfb = pd.read_json('C:/Users/patrick.walsh/workspace/GoodreadsScraper-1/book_.jl', lines=True)
dfr = pd.read_json('C:/Users/patrick.walsh/workspace/GoodreadsScraper-1/review_.jl', lines=True)
dfa = pd.read_json('C:/Users/patrick.walsh/workspace/GoodreadsScraper-1/author_.jl', lines=True)

# set todays date for relative date calculations
todays_date = datetime.date.today()

Book Transformation

In [2]:
# add prefix of URL to the URLs in Review dataset
url_prefix = 'https://www.goodreads.com'
dfr['url_full'] = url_prefix + dfr['url']

# join Review and Book datasets on URL
dfbr = pd.merge(dfb, dfr, left_on='url', right_on='url_full', how='left')

# create copy of index field
dfbr['id'] = dfbr.index

In [3]:
# clean shelf values
shelf_dict = {"read" : 'Read', "currently-reading" : 'Currently Reading', "to-read": 'To Read'}
dfbr=dfbr.replace({"shelf": shelf_dict})
dfbr['shelf']

# create empty lists to be set as new dataframe columns after the loop
award_ct = []
part_of_series = []
one_star_ratings = []
two_star_ratings = []
three_star_ratings = []
four_star_ratings = []
five_star_ratings = []
ratio_a = []
ratio_b = []
my_rating_int = []
rating_dif = []
publish_date_new = []
my_review_date_added_new = []
my_review_date_read_new = []
publish_decade = []

# create rating text to int dictionary
rating_dict = {'did not like it': '1',
                   'it was ok': '2',
                   'liked it': '3',
                   'really liked it': '4',
                   'it was amazing': '5'}

# create bins for average community rating
bins = [0, .5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]
dfbr['comm_rating_bin'] = pd.cut(dfbr['avg_rating'], bins, labels=['<.05','.5-.99','1-1.49','1.5-1.99','2-2.49','2.5-2.99','3-3.49','3.5-3.99','4-4.49','>4.5'], right=False)

# loop through every row in the Book Reviews (dfbr) dataframe to add values to the empty lists
for i,r in dfbr.iterrows():
    # count awards
    if str(r['awards'])=='nan':
        award_ct.append(None)
    else:
        award_ct.append(len(r['awards']))

    # part of series
    if pd.isna(r['series']):
        part_of_series.append('Standalone')
    else:
        part_of_series.append('Part of a series')

    # column for each of the rating counts
    one_star_ratings.append(dfbr['rating_histogram'][i]['1'])
    two_star_ratings.append(dfbr['rating_histogram'][i]['2'])
    three_star_ratings.append(dfbr['rating_histogram'][i]['3'])
    four_star_ratings.append(dfbr['rating_histogram'][i]['4'])
    five_star_ratings.append(dfbr['rating_histogram'][i]['5'])

    # ratio of 5 star ratings to 1 star ratings
    if one_star_ratings[i]:
        ratio_a.append(five_star_ratings[i]/one_star_ratings[i])
    else:
        ratio_a.append(None)

    # ratio of 5 and 4 star ratings to 1 and 2 star ratings
    if one_star_ratings[i]+two_star_ratings[i]:
        ratio_b.append((four_star_ratings[i]+five_star_ratings[i])/(one_star_ratings[i]+two_star_ratings[i]))
    else:
        ratio_b.append(None)

    # my rating text to int -- map values using the dictionary
    if r['my_rating']:
        my_rating_str = str(r['my_rating'])
        my_rating_int.append(rating_dict.get(my_rating_str))
    else:
        my_rating_int.append(None)

    # my rating versus average rating
    if my_rating_int[i]:
        my_rating_minus_avg = int(my_rating_int[i]) - r['avg_rating']
        rating_dif.append(my_rating_minus_avg)
    else:
        rating_dif.append(None)

    # remove ' 00:00:00' from publish_date
    if type(r['publish_date']) != float and str(r['publish_date'][:10]) != '0001-01-01':
        date_only = str(r['publish_date'][:10])
        publish_date_new.append(datetime.datetime.strptime(date_only, '%Y-%m-%d').date())
    else:
        publish_date_new.append(None)

    # remove ' 00:00:00' from my_review_date_added
    if type(r['my_review_date_added']) != float:
        date_only = str(r['my_review_date_added'][:10])
        my_review_date_added_new.append(datetime.datetime.strptime(date_only, '%Y-%m-%d').date())
    else:
        my_review_date_added_new.append(None)

    # remove ' 00:00:00' from my_review_date_read
    if type(r['my_review_date_read']) != float:
        date_only = str(r['my_review_date_read'][:10])
        my_review_date_read_new.append(datetime.datetime.strptime(date_only, '%Y-%m-%d').date())
    else:
        my_review_date_read_new.append(None)

# separate loop to obtain publish year and decade, as well as relative read year
dfbr['publish_date_new'] = publish_date_new
dfbr['publish_year'] = pd.DatetimeIndex(dfbr['publish_date_new']).year
dfbr['my_review_date_read_new'] = my_review_date_read_new
dfbr['read_year'] = pd.DatetimeIndex(dfbr['my_review_date_read_new']).year
rel_read_year = []

for i,r in dfbr.iterrows():
    if r['publish_year'] != None and str(r['publish_year']) != 'nan':
        pub_year_str = str(r['publish_year'])[0:3] + '0s'
        publish_decade.append(pub_year_str)
    else:
        publish_decade.append(None)
    
    # relative read year
    if r['read_year'] == todays_date.year:
        rel_read_year.append('this year')
    elif r['read_year'] == (todays_date.year - 1):
        rel_read_year.append('last year')
    else:
        rel_read_year.append(None)

# set newly populated lists to new columns in the dataframe
dfbr['award_ct'] = award_ct
dfbr['part_of_series'] = part_of_series
dfbr['1_star_ratings'] = one_star_ratings
dfbr['2_star_ratings'] = two_star_ratings
dfbr['3_star_ratings'] = three_star_ratings
dfbr['4_star_ratings'] = four_star_ratings
dfbr['5_star_ratings'] = five_star_ratings
dfbr['5_to_1_ratio'] = ratio_a
dfbr['4_5_to_1_2_ratio'] = ratio_b
dfbr['my_rating_int'] = my_rating_int
dfbr['rating_dif'] = rating_dif
dfbr['my_review_date_added_new'] = my_review_date_added_new
dfbr['publish_decade'] = publish_decade
dfbr['rel_read_year'] = rel_read_year

Author Tranformation

In [4]:
birth_date_new = []
death_date_new = []
birthplace_new = []

for i,r in dfa.iterrows():
    # remove timestamp for birth and death date
    if type(r['birth_date']) != float:
        date_only = str(r['birth_date'][:10])
        birth_date_new.append(datetime.datetime.strptime(date_only, '%Y-%m-%d').date())
    else:
        birth_date_new.append(None)

    if type(r['death_date']) != float:
        date_only = str(r['death_date'][:10])
        death_date_new.append(datetime.datetime.strptime(date_only, '%Y-%m-%d').date())
    else:
        death_date_new.append(None)

    # birthplace -- remove "in " where applicable
    if type(r['birthplace']) != float:
        if r['birthplace'][:3] == 'in ':
            birthplace_new.append(r['birthplace'][3:])
        else:
            birthplace_new.append(r['birthplace'])
    else:
        birthplace_new.append(None)

dfa['birth_date_new'] = birth_date_new
dfa['death_date_new'] = death_date_new
dfa['birthplace_new'] = birthplace_new

Community Book Rating Creation

In [5]:
dfc = dfbr[['id', '1_star_ratings',
       '2_star_ratings', '3_star_ratings', '4_star_ratings', '5_star_ratings']]
dfc = pd.melt(dfc, id_vars = ['id'], value_vars = ['1_star_ratings',
       '2_star_ratings', '3_star_ratings', '4_star_ratings', '5_star_ratings'])
dfc = dfc.replace({'1_star_ratings':1,
       '2_star_ratings':2, '3_star_ratings':3, '4_star_ratings':4, '5_star_ratings':5})
dfc['rating_prod'] = dfc['variable']*dfc['value']

Setting Creation

In [6]:
df_place = dfbr.explode('places')
dfs = df_place[['id','places']].drop_duplicates()[df_place['places'].isna() == False]

Output

In [7]:
dfbr = dfbr[['id','url_x', 'title', 'author', 'num_ratings', 'num_reviews', 'avg_rating',
       'num_pages', 'language', 'genres', 'awards',
       'characters', 'series',
       'isbn', 'isbn13', 'places', 'asin',
       'shelf', 'award_ct', 'part_of_series', '1_star_ratings',
       '2_star_ratings', '3_star_ratings', '4_star_ratings', '5_star_ratings',
       '5_to_1_ratio', '4_5_to_1_2_ratio', 'my_rating_int', 'publish_date_new',
       'my_review_date_added_new', 'my_review_date_read_new', 'rating_dif', 'img_url', 'publish_decade', 'rel_read_year', 'comm_rating_bin']]

dfa = dfa[['url', 'name', 'genres', 'avg_rating', 'num_reviews', 'num_ratings', 'about', 
        'influences', 'birth_date_new', 'death_date_new', 'img_url', 'birthplace_new']]

In [8]:
# Output datasets to csvs

## books/reviews
dfbr.to_csv('book_review.csv')

## authors
dfa.to_csv('author.csv')

## community ratings
dfc.to_csv('comm_rate.csv')

## settings
dfs.to_csv('settings.csv')